In [ ]:
!python -m spacy download en_core_web_lg

In [2]:
import os
import csv
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter

nlp = spacy.load("en_core_web_lg")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
path = "/content/data"
dir_list = os.listdir(path)
os.chdir("/content/data")

universal_new_lines = '\n' + '\r' + '\r\n' + '\v' + '\x0b' + '\f' + '\x0c' + '\x1c' + '\x1d' + '\x1e' + '\x85' + '\u2028' + '\u2029' 
pun = string.punctuation + '\t' + '—' + '‘’' + universal_new_lines
label_list = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'LAW', 'DATE', 'TIME', 'MONEY']
fields = ['VERB', 'PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'LAW', 'DATE', 'TIME', 'MONEY']
processed_data = []

In [4]:
def data_processing(data):
  # removing punctuations and new lines and tabs
  for sym in pun:
    cnt = data.count(sym)
    data = data.replace(sym, '', cnt)

  # lower the data
  data = data.lower()
  doc = nlp(data)

  token_classification = {'VERB':[], 'PERSON':[], 'NORP':[], 'FAC':[], 'ORG':[], 'GPE':[], 'LOC':[], 'PRODUCT':[], 'EVENT':[], 'LAW':[], 'DATE':[], 'TIME':[], 'MONEY':[]}

  # fetching entities and keywords
  for ent in doc.ents:
    if ent.label_ in label_list:
      if ent.text not in token_classification[ent.label_]:
        token_classification[ent.label_].append(ent.text)
  
  for token in doc:
    if token.pos_ == 'VERB':
      if token.lemma_ not in token_classification['VERB']:
        token_classification['VERB'].append(token.lemma_)

  return token_classification

In [5]:
for fname in dir_list:
  file = open(fname)
  content_list = file.readlines()
  while '\n' in content_list:
    content_list.remove('\n')
  for content in content_list:
    token_classification = data_processing(content)
    processed_data.append(token_classification)
# print(processed_data)

with open("/content/ProcessedData.csv", 'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = fields)
  writer.writeheader()
  writer.writerows(processed_data)